In [1]:
# Tutorial for setting up a small RAG system using Faiss 
# and evaluating it using the Gemini Flash 1.5 LLM and the DeepEval library
# I use the Google Gemini API (free tier, local API key), but DeepEval is compatible with several LLMs
# Google Gemini: https://ai.google.dev/gemini-api/docs/models/gemini
# DeepEval: https://docs.confident-ai.com/docs/guides-rag-evaluation

# Testing out this notebook with the latest version of DeepEval: v2.2.7
# Thankfully, not much had changes for the updates in DeepEval to break my code;
# Instead, the only additional parsing needed was on the output returned from an evaluation

# DeepEval v1.1.6 was fairly compatible with Google Gemini by creating a new LLM class that inherited from DeepEvalBaseLLM
# and adding methods that called Gemini's generation functions; it was a similar setup for the Embeddings, inheriting from DeepEvalBaseEmbeddingModel
# The only trick is that the LLM output needs to be in JSON format
# I used the pydantic and instructor libraries for this; the following gives good examples of how to use them
# Tutorial on using custom LLMs with DeepEval: https://docs.confident-ai.com/docs/guides-using-custom-llms

# Metrics available in DeepEval:
# - Contextual Precision: Evaluates whether the reranker in your retriever ranks more relevant nodes in your retrieval context higher than irrelevant ones.
# - Contextual Recall: Evaluates whether the embedding model in your retriever is able to accurately capture and retrieve relevant information based on the context of the input.
# - Contextual Relevance: Evaluates whether the text chunk size and top-K of your retriever is able to retrieve information without much irrelevancies.
# - Answer Relevancy: Evaluates whether the prompt template in your generator is able to instruct your LLM to output relevant and helpful outputs based on the retrieval_context.
# - Faithfulness: Evaluates whether the LLM used in your generator can output information that does not hallucinate AND contradict any factual information presented in the retrieval_context.
# - Other metrics are available for non-RAG systems; custom metrics can also be created (I did not test this)

In [19]:
# Set up environment

In [1]:
import os
from IPython.display import display, Markdown
import pandas as pd
from typing import List
from pydantic import BaseModel, ConfigDict # for JSON output from DeepEval
import instructor # for JSON output from DeepEval

# Replace these two Google Gemini imports with imports for your LLM
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# also need pip install llama-index-embeddings-langchain, added the below
from llama_index.embeddings.langchain import LangchainEmbedding

from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext, load_index_from_storage
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

import deepeval
from deepeval.models import DeepEvalBaseLLM, DeepEvalBaseEmbeddingModel
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset
from deepeval.synthesizer import Synthesizer
from deepeval import evaluate
from deepeval.evaluate import TestResult, print_test_result
from deepeval.metrics import (
    AnswerRelevancyMetric,
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    FaithfulnessMetric
)
#from deepeval.metrics.ragas import (
#    RagasMetric,
#    RAGASAnswerRelevancyMetric,
#    RAGASFaithfulnessMetric, 
#    RAGASContextualRecallMetric,
#    RAGASContextualPrecisionMetric,
#    RAGASContextualRelevancyMetric
#) 

/home/amber/anaconda3/envs/deepeval_test/lib/python3.11/site-packages/deepeval/__init__.py:52: UserWarning: You are using deepeval version 2.2.7, however version 2.3.3 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [2]:
# Environmental variable to opt out of DeepEval tracking telemetry data
os.environ["DEEPEVAL_TELEMETRY_OPT_OUT"] = "YES"

In [3]:
deepeval.telemetry_opt_out()

True

In [4]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [5]:
# set up local API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [6]:
# Establish RAG pipeline with Gemini

In [7]:
# # Create a Faiss vector store for RAG
# # If you already have an index created, skip a few coding cells to the LLM / embeddings setup

# # Example of creating a small vector store
# # Using 4 State of the Union speeches, all text from whitehouse.gov briefing room speeches posted online, edited to include a title with the date of the speech
# # Example from 2024:
# # https://www.whitehouse.gov/briefing-room/speeches-remarks/2024/03/07/remarks-of-president-joe-biden-state-of-the-union-address-as-prepared-for-delivery-2/

# # load and parse files
# sotu = []
# newfiles = ["./Speeches/titleedits/state_of_the_union_042921.txt", "./Speeches/titleedits/state_of_the_union_030122.txt", "./Speeches/titleedits/state_of_the_union_020723.txt", "./Speeches/titleedits/state_of_the_union_030724.txt"]
# for i in newfiles:
#     with open(i) as file:
#         for line in file:
#             nl = line.rstrip()
#             if nl != '':
#                 sotu.append(nl)

# # convert into Document format
# documents = [Document(text=line) for line in sotu]

In [9]:
# # Example of a loaded Document line
# documents[-1]

Document(id_='833ea164-b547-46ec-8854-cefdc83fbb10', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='May God protect our troops.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [10]:
# # Set up the faiss index
# d = 768 # dimensions of the input vector of the embedding model that we're going to use; in this case, the google embedding model
# faiss_index = faiss.IndexFlatL2(d)
# print(faiss_index.is_trained) # double check that the training worked

True


In [6]:
# Set up the llm, embeddings, and Settings for Faiss 
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # Replace with your LLM
doc_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") # Replace with your embeddings model
Settings.embed_model = doc_embeddings # used for LlamaIndex FaissVectorStore
Settings.llm = llm # used for LlamaIndex FaissVectorStore

In [13]:
# # Uncomment for when you need to re-embed and vectorize documents

# vector_store = FaissVectorStore(faiss_index=faiss_index)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, show_progress=True
# )

# # Save index to disk
# index.storage_context.persist()

# # Save/remember index id for loading next time
# index.index_id

In [7]:
# After you have a saved index, load that index for RAG answer generation:

# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
# My local index id '3d3c99c5-aa1c-42d7-a9ce-c4bb12fbc6d5' uses the 4 speeches including a title that includes the date it was given
# My local index id '95634851-570e-454e-983f-6634eeb72aee' contains 3200 documents from the rag_mini_wikipedia dataset
index = load_index_from_storage(storage_context=storage_context, index_id='95634851-570e-454e-983f-6634eeb72aee')

In [13]:
# # Optional- if you'd like to query your index
# # Set up query and chat engines with the index
# query_engine = index.as_query_engine(similarity_top_k=10)
# chat_engine = index.as_chat_engine(similarity_top_k=10, chat_mode='context')

In [ ]:
# # Example query and response
# query = "In detail, what has the President done to improve the economy over the four years of his speeches?"
# response = query_engine.query(query) 
# print(response.response)

In [ ]:
# # Get ranked scores for top k RAG source nodes
# for node in response.source_nodes:
#     print(f"{node.get_score()} -> {node.text}")

In [17]:
# # Example of using the chat engine with our index
# query = "You are an expert speech analyst and specialize in analyzing Presidential State of the Union speeches. Could you please analyze the speeches and generate 2 questions and answers from each speech, providing the document filename of each speech that relates to each question?"
# response = chat_engine.chat(query) 
# print(response.response)

In [ ]:
# # Optional: View chat history
# chat_engine.chat_history

In [12]:
# Code for DeepEval RAG evaluation library to work with Gemini and our local RAG setup

# In DeepEval v1.1.6 and in v2.2.7, an example Input/Output/Context is a 'LLMTestCase'
# You can evaluate LLMTestCases individually or in a large batch with the evaluate function
# Metric scores are produced as well as an LLM generated explanation for a given score 

# DeepEval can also synthetically generate data from documents - see Synthesizer and generate_goldens_from_docs, below. 
# Of the 3 libraries I tested, this function produced the most human-realistic queries.
# DeepEval also offers other functionality besides RAG evaluation, including "red teaming LLM applications for security vulnerabilities"

# https://docs.confident-ai.com/docs/guides-rag-evaluation

In [6]:
# DeepEval requires a json response. In practice, this has led to malformed json returned from the llm, even with as simple of a schema as this, 
# but this and the LLM class can likely be refined to improve responses
class Response(BaseModel):
    response: str

In [7]:
# Non Open-AI usage requires a custom LLM class for using DeepEval
# Tutorial with example code on using custom LLMs with DeepEval: https://docs.confident-ai.com/docs/guides-using-custom-llms
class CustomGeminiFlash(DeepEvalBaseLLM):
    def __init__(self):
        self.model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")
        model_config  = ConfigDict(protected_namespaces=())

    def load_model(self):
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel: 
        client = self.load_model()
        instructor_client = instructor.from_gemini(
            client=client,
            mode=instructor.Mode.GEMINI_JSON,
        )
        resp = instructor_client.messages.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            response_model=schema,
        )
        return resp

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return "Gemini 1.5 Flash"

In [8]:
# Similarly, a custom embedding model class is required for non Open-AI embeddings
# Tutorial on using custom embeddings with DeepEval: https://docs.confident-ai.com/docs/guides-using-custom-embedding-models 
class CustomGeminiEmbeddingModel(DeepEvalBaseEmbeddingModel):
    def __init__(self):
        model_config  = ConfigDict(protected_namespaces=())

    def load_model(self):
        return GoogleGenerativeAIEmbeddings(
            model="models/text-embedding-004"
        )

    def embed_text(self, text: str) -> List[float]:
        embedding_model = self.load_model()
        return embedding_model.embed_query(text)

    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        embedding_model = self.load_model()
        return embedding_model.embed_documents(texts)

    async def a_embed_text(self, text: str) -> List[float]:
        embedding_model = self.load_model()
        return await embedding_model.aembed_query(text)

    async def a_embed_texts(self, texts: List[str]) -> List[List[float]]:
        embedding_model = self.load_model()
        return await embedding_model.aembed_documents(texts)

    def get_model_name(self):
        "Custom Gemini Embeddings"

In [9]:
# Create custom llm and embeddings
custom_geminiflash = CustomGeminiFlash()
custom_geminiembeddings = CustomGeminiEmbeddingModel()

In [ ]:
# Generate a synthetic dataset of "Goldens" (AKA a dataset with 'input', 'context', 'source_file' columns -- not 'Retrieval_Context') with DeepEval
dataset = EvaluationDataset()
synthesizer = Synthesizer(model=custom_geminiflash, embedder=custom_geminiembeddings)
dataset.generate_goldens_from_docs(
    synthesizer=synthesizer,
    document_paths=['Speeches/titleedits/state_of_the_union_042921.txt', 'Speeches/titleedits/state_of_the_union_030122.txt', 
                    'Speeches/titleedits/state_of_the_union_020723.txt', 'Speeches/titleedits/state_of_the_union_030724.txt'],
    max_goldens_per_document=3, # maximum number of questions to generate per document
    include_expected_output=True
)

dataset.save_as(file_type="csv", directory=".")

In [13]:
# Example of evaluating one example/test case

# Parameters in a DeepEval LLMTestCase:
# Input: Question/query for the LLM
# Actual Output: Answer returned from the LLM
# Expected Output: The ideal output for the input/question
# Retrieval Context (optional): LLM's actual retrieval results from the RAG system
# Context (optional): Additional ground truth context besides RAG

contextual_precision = ContextualPrecisionMetric(model=custom_geminiflash)
contextual_recall = ContextualRecallMetric(model=custom_geminiflash)
contextual_relevancy = ContextualRelevancyMetric(model=custom_geminiflash)

test_case = LLMTestCase(
    input="What measures does the speaker propose to lower prescription drug costs in America?",
    actual_output="The speaker proposes giving Medicare the power to negotiate lower prescription drug prices, capping prescription drug costs at $2,000 a year for everyone, and allowing Medicare to negotiate lower prices for 500 drugs over the next decade.s",
    expected_output="The speaker proposes that Medicare should be given the power to negotiate lower drug prescription prices. They argue that this would save hundreds of billions of dollars and lower prescription drug costs for everyone. The speaker also states that the money saved could be used to strengthen the Affordable Care Act and expand Medicare coverage benefits without costing taxpayers an additional penny.",
    retrieval_context=['Let’s do what we’ve always talked about for all the years I was down here in this — in this body — in Congress.  Let’s give Medicare the power to save hundreds of billions of dollars by negotiating lower drug prescription prices.  (Applause.)', 'In fact, we pay the highest prescription drug prices of anywhere in the world right here in America — nearly three times — for the same drug, nearly three times what other countries pay.  We have to change that, and we can.', 'And we’re finally giving Medicare the power to negotiate drug prices. Bringing down prescription drug costs doesn’t just save seniors money.', 'For years people have talked about it but I finally got it done and gave Medicare the power to negotiate lower prices for prescription drugs just like the VA does for our veterans.', 'And, by the way, that won’t just — that won’t just help people on Medicare; it will lower prescription drug costs for everyone.', 'Now I want to cap prescription drug costs at $2,000 a year for everyone!', 'We know how to do this.  The last President had that as an objective.  We all know how outrageously expensive drugs are in America.', 'Make no mistake, if you try to do anything to raise the cost of prescription drugs, I will veto it.', 'Now it’s time to go further and give Medicare the power to negotiate lower prices for 500 drugs over the next decade.', 'It will cut the federal deficit, saving tax payers hundreds of billions of dollars on the prescription drugs the government buys for Medicare.']
)

In [23]:
# For RAG systems, DeepEval recommends the following Retrieval and Generation metrics:
# Retrieval metrics:
contextual_precision.measure(test_case)
print("Contextual Precision Score: ", contextual_precision.score)
print("Contextual Precision Reason: ", contextual_precision.reason)

contextual_recall.measure(test_case)
print("Contextual Recall Score: ", contextual_recall.score)
print("Contextual Recall Reason: ", contextual_recall.reason)

contextual_relevancy.measure(test_case)
print("Contextual Relevancy Score: ", contextual_relevancy.score)
print("Contextual Relevancy Reason: ", contextual_relevancy.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Contextual Precision Score:  0.9095238095238096
Contextual Precision Reason:  The score is 0.91 because the first five nodes are relevant and directly address the speaker's proposal to lower drug prices through Medicare negotiation.  However, the sixth node, focusing on capping prescription drug costs,  is a separate proposal and doesn't directly align with the initial proposal, making it a relevant node ranked lower. The seventh and eighth nodes don't mention specific measures, making them less relevant compared to the first five nodes that directly discuss the speaker's proposed measure.


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Contextual Recall Score:  1.0
Contextual Recall Reason:  The score is 1.00 because the speaker proposes that Medicare should be given the power to negotiate lower drug prescription prices, and the node(s) in retrieval context) support this with phrases like 'Let’s give Medicare the power to save hundreds of billions of dollars by negotiating lower drug prescription prices.'.


Contextual Relevancy Score:  0.5
Contextual Relevancy Reason:  The score is 0.50 because the reasons for irrelevancy indicate that the context doesn't specifically outline measures to lower prescription drug costs, but it does discuss the speaker's position and impact on costs. "The context discusses the high prescription drug prices in America and suggests changing them, but it doesn't mention any specific measures the speaker proposes to lower costs." and "The context only states the speaker's position on raising the cost of prescription drugs, not any measures to lower it." highlight this, suggesting a partial relevance as the context doesn't ignore the topic entirely.


In [29]:
# Generation metrics:
answer_relevancy = AnswerRelevancyMetric(model=custom_geminiflash)
faithfulness = FaithfulnessMetric(model=custom_geminiflash)
                                 
answer_relevancy.measure(test_case)
print("Answer Relevancy Score: ", answer_relevancy.score)
print("Answer Relevancy Reason: ", answer_relevancy.reason)

faithfulness.measure(test_case)
print("Faithfulness Score: ", faithfulness.score)
print("Faithfulness Reason: ", faithfulness.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Answer Relevancy Score:  1.0
Answer Relevancy Reason:  The score is 1.00 because the input asks for measures to lower prescription drug costs and the provided JSON correctly represents that information. Great job!


Faithfulness Score:  1.0
Faithfulness Reason:  The score is 1.00 because there are no contradictions, this is great!


In [10]:
# Example of measuring metrics for multiple test cases / a full dataset

# Load manually curated dataset
evaldataset = EvaluationDataset()
evaldataset.add_test_cases_from_csv_file(
    file_path="datasets/rag_mini_wikipedia_complete_chat_ares.csv",
    input_col_name="question",
    actual_output_col_name="answer",
    expected_output_col_name="ground_truth",
#    context_col_name="context",
#    context_col_delimiter= ",",
    retrieval_context_col_name="contexts", # Context that the LLM produced when it answered the Input
    retrieval_context_col_delimiter= ","
#    additional_metadata_col_name="source_file"
)

In [11]:
# Retriever metrics:
contextual_precision = ContextualPrecisionMetric(model=custom_geminiflash, include_reason=False) # include_reason=False is an option
contextual_recall = ContextualRecallMetric(model=custom_geminiflash)
contextual_relevancy = ContextualRelevancyMetric(model=custom_geminiflash) # Note: this was the only metric that would not finish execution for the manually curated dataset (too many 429 errors)

# Generation metrics:
answer_relevancy = AnswerRelevancyMetric(model=custom_geminiflash)
faithfulness = FaithfulnessMetric(model=custom_geminiflash, include_reason=False)

In [12]:
len(evaldataset.test_cases)

903

In [13]:
all_1results = []

In [22]:
#pd.DataFrame(contextprecision_results).to_clipboard(index=False,header=False)
len(all_1results)

0

In [50]:
# 2 Options for Metrics Evaluation: 

# 1) Iterating through test cases seems to work better than bulk evaluation with evaluate,
# as errors encountered with evaluate(...) cause no results to be returned
# Looping at least saves partial results until an error occurs
# I encountered this sometimes with contextual_relevancy and contextual_precision on the test dataset (429 errors or Invalid JSON errors),
# yet typically I got results if I iterated through individual test_cases
# For future: https://github.com/confident-ai/deepeval/issues/964 may assist with incorrect json errors like what was being returned

# Example for evaluating one test case at a time
# contextprecision_results = []
for i in range(0, 1): # testing first, typical:  # range(0, len(evaldataset.test_cases)
    print(i)
    eval_1metric = evaluate(test_cases=[evaldataset.test_cases[i]], metrics=[contextual_precision], throttle_value=10) #throttle_value is for rate limiting, in seconds between queries; 10 for rate limits on gemini 1.5, 20 for gemini 2.0 exp
    all_1results.append(eval_1metric.test_results[0]) # edit for newer (2.2.7) version of DeepEval

# 2) Evaluate through test_cases in bulk
# In testing, at least faithfulness, contextual_precision metrics worked this way with a small, manually curated dataset

# Throttle_value is for rate limiting- in seconds between queries
# test_precision = evaluate(test_cases=evaldataset.test_cases, metrics=[contextual_precision], throttle_value=90)

250


✨ You're running DeepEval's latest Contextual Precision Metric! (using Gemini 1.5 Flash, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |███████████████████████████████|100% (1/1) [Time Taken: 00:20, 20.03s/test case]



Metrics Summary

  - ✅ Contextual Precision (score: 0.625, threshold: 0.5, strict: False, evaluation model: Gemini 1.5 Flash, reason: None, error: None)

For test case:

  - input: What positions Ford played in the school football team?
  - actual output: Gerald Ford played center and linebacker for the University of Michigan football team.

  - expected output: Center and linebacker
  - context: []
  - retrieval context: ["['Ford attended Grand Rapids South High School and was a star athlete and captain of his football team. In 1930", " he was selected to the All-City team of the Grand Rapids City League. He also attracted the attention of college recruiters.'", " 'At Michigan", ' Ford became a member of the Delta Kappa Epsilon fraternity and washed dishes at his fraternity house to earn money for college expenses. Following his graduation in 1935 with a degree in political science and economics', ' he turned down contract offers from the Detroit Lions and Green Bay Packers of the N

...

✨ You're running DeepEval's latest Contextual Precision Metric! (using Gemini 1.5 Flash, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |███████████████████████████████|100% (1/1) [Time Taken: 00:24, 24.70s/test case]



Metrics Summary

  - ✅ Contextual Precision (score: 1.0, threshold: 0.5, strict: False, evaluation model: Gemini 1.5 Flash, reason: None, error: None)

For test case:

  - input: Are Gray Wolves native to North America?
  - actual output: While Gray Wolves are currently found in North America, the text indicates they migrated there from Eurasia, meaning they are not *native* in the sense of having originated there.

  - expected output: Yes
  - context: []
  - retrieval context: ["['Current theories propose that the gray wolf first evolved in Eurasia during the early Pleistocene. The rate of changes observed in DNA sequence date the South-East Asiatic lineage to about 800", '000 years', ' as opposed to the American and European lineages which stretch back only 150', '000.    The gray wolf then migrated into North America from the Old World', ' probably via the Bering land bridge (that once joined Alaska and Siberia)', ' around 400', '000 years ago.  The gray wolf then coexisted with 

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.  
‼️ Looking for a place for your LLM test data to live 🏡? Use Confident AI for test reports, benchmark analysis, 
compare models/prompts, and catch regressions for your LLM system.

In [51]:
len(all_1results)

300

In [52]:
# Quick print for results after evaluation of single test cases, as needed
# Get scores to calculate average score
scores = []
for i in [all_1results]:
    for j in i:
        if type(j) == TestResult:
            scores.append(j.metrics_data[0].score)
            #print_test_result(j)
        else:
            print_test_result(j[0])

In [53]:
# Calculate the average for the metric (with 918 examples total)
scoredata = pd.DataFrame(scores, index=None)
scoredata.mean()

0    0.823368
dtype: float64

In [54]:
len(scoredata)

300

In [45]:
len(all_1results)

333

In [46]:
# to copy contents of all_results to the clipboard as needed
pd.DataFrame(all_1results).to_clipboard(index=False,header=False)
# all_results

In [47]:
# Save the scores
scoredata.to_csv("results/deepeval_updatedlib_contextualprecision_gemini_15_rag_mini_wikipedia_0_300.csv", index=False)

In [1]:
import os
from IPython.display import display, Markdown
import pandas as pd

In [39]:
# Calculating deepeval gemini 2.0 faithfulness
data1 = pd.read_csv("results/deepeval_faithfulness_gemini_2_rag_mini_wikipedia_0_121.csv", index_col=None)
data2 = pd.read_csv("results/deepeval_faithfulness_gemini_2_rag_mini_wikipedia_121_596.csv", index_col=None)
data3 = pd.read_csv("results/deepeval_faithfulness_gemini_2_rag_mini_wikipedia_596_903.csv", index_col=None)

In [40]:
alldata2 = pd.concat([data1, data2, data3], ignore_index=True)

In [41]:
alldata2

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
898,1.0
899,1.0
900,1.0
901,1.0


In [43]:
alldata2.mean()

0    0.967882
dtype: float64

In [44]:
alldata2.to_csv("results/deepeval_faithfulness_gemini_2_rag_mini_wikipedia_903.csv", index=False)

In [15]:
refinedscoredata = scoredata[:903]

In [18]:
# Calculate the average for the metric
refinedscoredata.mean()

0    0.792454
dtype: float64

In [17]:
# Save the refined scores
refinedscoredata.to_csv("results/deepeval_contextprecision_rag_mini_wikipedia_903.csv", index=False)

In [26]:
# DeepEval also has the RAGAS metrics available for evaluation

# Unforuntately the RAGAS metrics in DeepEval only accept langChain chat models, so the Gemini DeepEvalBaseLLM class will not work with these metrics
# Need to use our LangChain LLM created earlier:  
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# doc_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") 

ragasmetric = RagasMetric(model=llm, embeddings=doc_embeddings)
ragas_ar = RAGASAnswerRelevancyMetric(model=llm, embeddings=doc_embeddings)
ragas_f = RAGASFaithfulnessMetric(model=llm)
ragas_crecall = RAGASContextualRecallMetric(model=llm)
ragas_cp = RAGASContextualPrecisionMetric(model=llm)
ragas_crel = RAGASContextualRelevancyMetric(model=llm) # Note: This metric did not work in testing; returned errors related to the model

In [ ]:
# Example of evaluation for RagasMetric, an average of RAGAS's Answer Relevancy, Faithfulness, Contextual Recall, and Contextual Precision metrics
eval_ragas = evaluate(test_cases=[evaldataset.test_cases[0]], metrics=[ragasmetric], throttle_value=90)

In [ ]:
# Example of evaluation for each ragas metric individually
eval_ragas_f = evaluate(test_cases=evaldataset.test_cases, metrics=[ragas_f], throttle_value=90)